In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The Dataset
training = spark.read.csv('test2.csv',header=True,inferSchema=True)

In [3]:
training.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|Vaibhav| 26|         3| 30000|
|  Sunny| 30|         4| 35000|
|  Rohit| 25|         8| 50000|
|   Paul| 24|         2| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
| Mahesh| 24|         2| 40000|
+-------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

[Age,Experience]------> new feature-----> independent feature

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [10]:
output = featureassembler.transform(training)


In [11]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|Age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|Vaibhav| 26|         3| 30000|          [26.0,3.0]|
|  Sunny| 30|         4| 35000|          [30.0,4.0]|
|  Rohit| 25|         8| 50000|          [25.0,8.0]|
|   Paul| 24|         2| 20000|          [24.0,2.0]|
| Harsha| 21|         1| 15000|          [21.0,1.0]|
|Shubham| 23|         2| 18000|          [23.0,2.0]|
| Mahesh| 24|         2| 40000|          [24.0,2.0]|
+-------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data=output.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [26.0,3.0]| 30000|
|          [30.0,4.0]| 35000|
|          [25.0,8.0]| 50000|
|          [24.0,2.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
|          [24.0,2.0]| 40000|
+--------------------+------+



In [14]:
#inplement ML Algo 
from pyspark.ml.regression import LinearRegression
#train test split 
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor=regressor.fit(train_data)

In [16]:
### Coefficients
regressor.coefficients

DenseVector([1902.0979, 3636.3636])

In [17]:
#### Intercepts
regressor.intercept

-27174.825174827114

In [19]:
#prediction
pred_results=regressor.evaluate(test_data)

In [21]:
#prediction
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16405.594405594158|
|          [30.0,4.0]| 35000| 44433.56643356687|
+--------------------+------+------------------+



In [22]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(5419.580419580514, 45483935.644778565)